In [1]:
from pathlib import Path

data_path = Path.cwd().parent / "lionagi"  # Path to the data directory
data_path

PosixPath('/Users/lion/lionagi/lionagi')

In [2]:
# %pip installm llama_index

In [3]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.indices import VectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding

In [4]:
docs = SimpleDirectoryReader(
    data_path, recursive=True, required_exts=[".py"]
).load_data()
docs = [i for i in docs if len(i.text) > 100]

In [5]:
len(docs)

169

In [6]:
embed_model = OpenAIEmbedding(model="text-embedding-3-small")
index = VectorStoreIndex.from_documents(docs, embed_model=embed_model)

In [7]:
query = "what is the Branch object in lionagi, why is it used? How can we use Branch object to create a multi-agent HR automation framework? Please provide complete functional codes."

In [8]:
retrieve_engine = index.as_query_engine(top_k=5)

In [9]:
await retrieve_engine.aquery(query)

Response(response='The `Branch` object in lionagi is used as the active branch that orchestrates messages, models, and logs within the system. It is a key component for managing communication flow and data processing.\n\nTo create a multi-agent HR automation framework using the `Branch` object, you can define different agents for various HR tasks. Each agent can handle specific HR functions such as recruitment, onboarding, performance management, etc. Here is a simplified example of how you can structure the code:\n\n```python\nfrom lionagi.session.branch import Branch\n\nclass HRRecruitmentAgent:\n    def __init__(self, branch):\n        self.branch = branch\n\n    def recruit_employee(self, candidate_info):\n        instruct = Instruction(instruction="Recruit new employee", context=candidate_info)\n        response = self.branch.chat(instruct=instruct, invoke_actions=True)\n        return response\n\nclass HROnboardingAgent:\n    def __init__(self, branch):\n        self.branch = bra

In [10]:
# from lionagi.operations.compress.perplexity import compress_text
from lionagi import iModel

gpt4o = iModel(
    provider="openai",
    model="gpt-4o-2024-11-20",
)


async def query_lion(query: str):
    """
    Query the lionagi index with the given query.
    This function can be invoked in parallel.

    Args:
        query (str): The query to search for.

    Returns:
        list: A list of results.
    """
    result = await retrieve_engine.aquery(query)
    res = result.response
    res += " ".join([i.node.text for i in result.source_nodes])
    return res
    # return await compress_text(res, gpt4o, target_ratio=0.15)

In [11]:
from lionagi import Branch

branch = Branch(chat_model=gpt4o, tools=[query_lion])

In [12]:
instruction = "what is the Branch object in lionagi, why is it used? how does that relate to the rest of lionagi system? please Write a complete functional python porgram that uses lionagi to automate HR system, you should explore the codebase and provide a detailed explanation of the code as well."
guidance = "This is a complex problem, please explain in details and invoke the query tool for at least 10 queries, with 3-5 queries per round, use parallel tool calling to help you, queries should be unique, the final output should be extensive and feature rich."

In [13]:
from pydantic import BaseModel


class Codes(BaseModel):
    code: str
    comment: str


out = await branch.ReAct(
    instruct={
        "instruction": instruction,
        "guidance": guidance,
    },
    interpret=True,
    max_extensions=5,
    extension_allowed=True,
    response_format=Codes,
)

In [14]:
from IPython.display import Markdown

Markdown(branch.msgs.assistant_responses[0].response)

"Explain the purpose and functionality of the 'Branch' object in the LionAGI system, including its role and relationship to the rest of the LionAGI framework. Additionally, write a complete, functional Python program that uses LionAGI to automate an HR system. Explore the LionAGI codebase thoroughly and provide a detailed explanation of the program. Ensure the explanation is extensive and feature-rich. Use the query tool for at least 10 unique queries, with 3-5 queries per round, leveraging parallel tool calling to enhance efficiency."

In [15]:
branch.to_df()

,created_at,role,content,id,sender,recipient,metadata
0,2025-01-13 17:14:19.072930,user,{'context': ['User input: {'instruction': 'wha...,9a697bed-3a88-4fe5-9235-42ab4c6768e7,user,ddd78b10-c3cd-4674-b255-ee1fdcf68f20,{'lion_class': 'lionagi.protocols.messages.ins...
1,2025-01-13 17:14:21.216647,assistant,"{'assistant_response': '""Explain the purpose a...",ed9ef1f8-9675-4ef8-98cc-e7e421411b2d,ddd78b10-c3cd-4674-b255-ee1fdcf68f20,user,{'model_response': {'id': 'chatcmpl-ApMzLxnoJX...
2,2025-01-13 17:14:21.222819,user,"{'context': [], 'tool_schemas': {'tools': [{'t...",304615ad-dc94-43b6-a045-35e08dfaaa7d,user,ddd78b10-c3cd-4674-b255-ee1fdcf68f20,{'lion_class': 'lionagi.protocols.messages.ins...
3,2025-01-13 17:14:24.773014,assistant,"{'assistant_response': '```json { ""analysis""...",0571c0c1-fab1-4696-bc68-6978a65a3b07,ddd78b10-c3cd-4674-b255-ee1fdcf68f20,user,{'model_response': {'id': 'chatcmpl-ApMzNDkcFi...
4,2025-01-13 17:14:25.902668,action,"{'action_request': {'function': 'query_lion', ...",6c440917-5ef6-426b-9df1-123e7a7ca123,ddd78b10-c3cd-4674-b255-ee1fdcf68f20,f4a3e6de-5205-4267-a8a6-ccbaa9882be2,{'lion_class': 'lionagi.protocols.messages.act...
5,2025-01-13 17:14:25.902798,action,{'action_request_id': '6c440917-5ef6-426b-9df1...,3bac22a4-97f0-443e-a469-fed49c38e8b6,f4a3e6de-5205-4267-a8a6-ccbaa9882be2,ddd78b10-c3cd-4674-b255-ee1fdcf68f20,{'lion_class': 'lionagi.protocols.messages.act...
6,2025-01-13 17:14:26.187974,action,"{'action_request': {'function': 'query_lion', ...",ab8e0273-4294-4e6c-8bc6-cdba07681091,ddd78b10-c3cd-4674-b255-ee1fdcf68f20,f4a3e6de-5205-4267-a8a6-ccbaa9882be2,{'lion_class': 'lionagi.protocols.messages.act...
7,2025-01-13 17:14:26.188104,action,{'action_request_id': 'ab8e0273-4294-4e6c-8bc6...,7f8ec976-2d52-4de9-97f6-2775dee6c29d,f4a3e6de-5205-4267-a8a6-ccbaa9882be2,ddd78b10-c3cd-4674-b255-ee1fdcf68f20,{'lion_class': 'lionagi.protocols.messages.act...
8,2025-01-13 17:14:26.438228,action,"{'action_request': {'function': 'query_lion', ...",c7e5d19d-aa99-48a8-a368-a99d0993b349,ddd78b10-c3cd-4674-b255-ee1fdcf68f20,f4a3e6de-5205-4267-a8a6-ccbaa9882be2,{'lion_class': 'lionagi.protocols.messages.act...
9,2025-01-13 17:14:26.438369,action,{'action_request_id': 'c7e5d19d-aa99-48a8-a368...,29fac337-045a-4e96-a665-b5b8182d86ca,f4a3e6de-5205-4267-a8a6-ccbaa9882be2,ddd78b10-c3cd-4674-b255-ee1fdcf68f20,{'lion_class': 'lionagi.protocols.messages.act...


In [16]:
from IPython.display import Markdown, display


display(Markdown(f"```python\n{out.code}\n```"))
display(Markdown(f"{out.comment}"))

```python
from lionagi.session.branch import Branch
from lionagi.protocols.types import ActionRequest, ActionResponse

# Define HR automation use case as an asynchronous function
async def automate_hr_task(branch):
    """
    Function to automate an HR task (e.g., employee performance review).
    This demonstrates the use of LionAGI's Branch object.
    """
    # Example HR Task: Conduct Performance Review
    action_request = ActionRequest.create(
        function="performance_review",
        arguments={
            "employee_id": "E12345",
            "review_period": "2023-Q3"
        }
    )

    try:
        # Invoke the action through the Branch object
        response = await branch._act(action_request)

        # Process the result
        if response:
            print(f"Task '{response.function}' executed successfully.")
            print(f"Response: {response.output}")
        else:
            print("Task execution failed. No valid response.")

    except Exception as e:
        print(f"Error encountered: {e}")

# Instantiate the Branch object
hr_branch = Branch()

# Execute the automated HR task
import asyncio
asyncio.run(automate_hr_task(hr_branch))
```

The 'Branch' object in LionAGI serves as the primary orchestrator for managing conversations, tool invocations, logging, and structured responses. It plays a pivotal role in coordinating messages between different modules, ensuring seamless integration across workflows and enabling automation.

**Detailed Explanation:**
The provided Python program demonstrates the usage of LionAGI's 'Branch' class to automate an HR-related task. Here are the detailed steps:

1. **ActionRequest Creation:** An action request is defined with specific parameters (e.g., employee_id and review_period) to conduct a performance review. The `ActionRequest.create` method is used to construct this request with the required function and arguments.

2. **Action Execution via Branch:** The action request is passed to the `branch._act()` method, which handles the invocation of the specified HR function. The 'Branch' object manages underlying tool schemas, error handling, and communication with LionAGI's backend systems.

3. **Response Processing:** Based on the success of the action, the program either prints the result or handles errors gracefully for debugging purposes.

**Features and Best Practices:**
- **Orchestration:** The 'Branch' seamlessly handles communication between tools, LLMs, and logs, allowing efficient automation.
- **Error Handling:** Error conditions and exceptions are logged using in-built mechanisms, ensuring reliability.
- **Async Execution:** The entire workflow is implemented asynchronously, leveraging the non-blocking advantages of Python's `asyncio`.

This example can be extended to implement workflows such as onboarding, payroll processing, or leave approval, showcasing the flexibility of LionAGI in automating diverse HR processes.